In [1]:
%matplotlib qt
import numpy as np 
import matplotlib.pyplot as plt
from spec_im_utils import gui_fname, plot_pl_summary, plot_si_bands, plot_bss_results, plot_decomp_results
from spec_im import SpectralImage, PLSpectralImage, PL3DSpectralImage
import math

In [2]:
# fname = gui_fname()
fname = 'D:/Chris/BBOD_Share/uv_microscope/190430 confocal GaN pyramid uvpl/190426_183756_oo_asi_hyperspec_3d_scan.h5'

In [3]:
si = PL3DSpectralImage(fname=fname)

Load from D:/Chris/BBOD_Share/uv_microscope/190430 confocal GaN pyramid uvpl/190426_183756_oo_asi_hyperspec_3d_scan.h5 complete.
5 x 101 x 102 spatial x 1044 spectral points


In [4]:
si = si[354:600]
plot_pl_summary(si, num_rows=3);
print(si.spec_x_array.shape)

(316,)


In [19]:
si.decomposition(algorithm='svd', output_dimension=0.9999)

In [20]:
si.plot_explained_variance_ratio()

In [21]:
si.blind_source_separation(number_of_components=10, max_iter=2000)

array shape (10, 51510)
(10, 51510) (10, 5, 102, 101)


In [22]:
ii = 0
slice_list = si.get_slice_list()
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]

num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)
(nz, ny, nx, nf) = np.shape(si.spec_im)
num_cols = 3 + nz
show_scalebar=False

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:  
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        for ii in range(nz):
            plt.subplot(num_rows, num_cols, num_cols*kk + 1 + ii)
            if kk == 0:
                z_pos = 'z = %0.2f $\mu$m' % ((si.z_array[ii]-si.z_array[0])*1e3)
            else:
                z_pos = ''
                
            if kk==0 and ii==0:
                show_scalebar=True
            else:
                show_scalebar=False
            
            si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right',
                     title='%s' % z_pos, show_scalebar=show_scalebar)
            # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (num_cols*kk + nz + 2, num_cols*kk + nz + 3))
        si._plot_spec(factors[index,:])
        plt.title('%d' % index)
        

(10, 5, 102, 101) (10, 316) (316,)


Traceback (most recent call last):
  File "C:\CondaEnvs\hyperspy\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\drawing\utils.py", line 171, in function_wrapper
    function()
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\drawing\signal1d.py", line 144, in _on_close
    super(Signal1DFigure, self)._on_close()
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\drawing\figure.py", line 103, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\signal.py", line 2078, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plot),
  File "C:\CondaEnvs\hyperspy\lib\site-packages\hyperspy\events.py", line 373, in disc

In [9]:
slice_list = si.get_slice_list()
plt.figure()
si.get_slice(slice_list[3])[330:360].plot()

In [ ]:
mask = si.get_slice(slice_list[3])[330:360].spec_im.sum(axis=-1) < 7.95e4
print(mask.shape)
plt.figure()
plt.imshow(mask)
si.apply_mask(mask)

In [ ]:
# mask2 = np.empty(np.shape(si.spec_im.sum(axis=-1)))
# mask2[kk in range(np.size(si.z_array)), :, :, :] = mask
# #si.apply_mask(mask2)
# print('mask size', np.size(mask2), 'nonzero values in mask', np.count_nonzero(mask2.flatten()), '', np.size(si.spec_im.sum(axis=-1)))

In [16]:
si.decomposition(algorithm='svd', output_dimension=0.9999)
si.plot_explained_variance_ratio()

In [17]:
si.blind_source_separation(number_of_components=10)

array shape (10, 51510)
(10, 51510) (10, 5, 102, 101)


In [18]:
ii = 0
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]
num_cols = 3
num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        plt.subplot(num_rows, num_cols, 3*kk + 1)
        si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right')
        # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        si._plot_spec(factors[index,:])

(10, 5, 102, 101) (10, 316) (316,)


In [10]:
s, fsi = si.to_signal()

In [11]:
s.plot()

In [ ]:
plt.figure()
fsi.plot()

In [12]:
# perform principal component analysis, look at the explained variance
s.decomposition(algorithm='svd')
s.plot_explained_variance_ratio(n=None)

In [13]:
COMPS = 10

In [14]:
# take a look at orthogonalized 
plot_bss_results(s,fsi,title='BSS',cmap='viridis',fig_rows=5,components=COMPS,max_iter=2000)

[<Figure size 1000x1000 with 15 Axes>, <Figure size 1000x1000 with 15 Axes>]

In [ ]:
sc = s.get_decomposition_model(components=5)
sc.plot()

In [ ]:
(s-sc).plot()

In [ ]:
sc = s.get_decomposition_model(components=10)
sc.plot()

In [ ]:
s.plot_decomposition_results()

In [ ]:
s.plot_bss_results()

In [ ]:
sc = s.get_decomposition_model(components=4)
sc.plot()